In [1]:
%run ml_header.py

In [2]:
%run get_data.py
benchmarks = ('adpcm_encoder', 'aes', 'ann', 'average', 'decimation', 'fft_fixed', 'fir',
              'idct', 'interpolation', 'kasumi', 'qsort', 'snow3g', 'sobel')  # 13
gd = GetData('data/ASIC-2-FPGA', benchmarks)
gd.main()

In [3]:
%run methods.py
%run machine_learning.py

In [4]:
features = ['AREA', 'state', 'FU', 'REG', 'MUX', 'DEC', 'pin_pair',
            'net', 'max', 'min', 'ave', 'MISC', 'MEM', 'sim', 'Pmax',
            'Pmin', 'Pave', 'Latency', 'BlockMemoryBit', 'DSP', 'Slices']
invalid_features = ['Slices', 'Latency']
valid_features = [i for i in features if i not in invalid_features]
label = 'Slices'

In [28]:
estimator = ensemble.AdaBoostRegressor(tree.DecisionTreeRegressor())
# estimator = linear_model.LinearRegression()

In [39]:
train_set = [gd.data_v4[i][features] for i in ('adpcm_encoder', 'average', 'fir')]
train_set = pd.concat(train_set, axis=0, ignore_index=True)
# train_set = gd.data_v4['interpolation']

In [40]:
train_set = ML.fix_missing_data(train_set)
X, y = ML.separate_feature_label(train_set, valid_features=valid_features, label=label)
X = ML.feature_scaling(X)

In [41]:
estimator.fit(X, y)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=50,
         random_state=None)

In [42]:
benchmark_test = 'ann'
test_set = ML.fix_missing_data(gd.data_v4[benchmark_test][features])
X_test, y_test = ML.separate_feature_label(test_set, valid_features=valid_features, label=label)
X_test = ML.feature_scaling(X_test)

In [43]:
y_pred = estimator.predict(X_test)
test_set['AREA'] = y_pred
DirectMapping.main(test_set)

{'adrs_ave': 0.071588459912025129,
 'adrs_ave_rms': 0.17528274639195993,
 'adrs_max': 0.15411558669001751,
 'adrs_max_rms': 0.74192971063779689,
 'cardinality': 7,
 'dominance': 0.2222222222222222,
 'hypervolume': 0.82055907286697749}